### 論文: 實驗一

- 資料集: Dofloo 的 4 篇文章
- 內容: 衡量 Extraction 模組，抓關鍵字的準確度 (TP/FP)
- 內容: 衡量 Extraction 模組，抓物件的動詞的準確度 (TP/FP)

In [2]:
import re
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
import json
import pickle
from typing import Dict, List, get_type_hints
from IPython.display import display

''' 自己寫的模組 '''
# from cc_regex_script import RegexMatchResult, RegexMaster 
# from cc_nlp_script import find_verb_of_vocab, OperationMode, OperationEvaluator
# from ASG import AttackGraph, Node, Edge, FileTable #, build
# import Utility

' 自己寫的模組 '

In [3]:
asg_regex = ['.*/selinux', '/.*bin/sed', '/dev/urandom', '/etc/init\\.d/.*', '/etc/rc.*\\.d/.*', '/etc/rc\\.local', '/proc/net/dev', '/proc/self', '/proc/stat', '/sys/devices/.*', '/etc/sed[0-9a-zA-Z]{6}', 'Permission', 'Permission:[0-9]{3}', '^GID.*', '^PID.*', '^UID.*', '^eth.*', '^sed$', '^sh$', '^uname$',]
print(len(asg_regex))

cntone_regex = ['0x[0-9a-zA-Z]{1,16}', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', ]
# 'port \\d+' 先移除

20


In [11]:
family = 'Dofloo'
report_csv_path = '../../C parse report/sentence csvs/'
report_fir_path = '../../C parse report/'
# family, samplename, outputfolder = 'Xorddos', '???', './result'
outputfolder = './result'

def get_all_filenames(dir: str='./') -> list:
    ''' traverse root directory, and list directories as dirs and files as files. Return filenames in rootdir. '''
    files = [f for f in listdir(dir) if isfile(join(dir, f))]
    files.sort()
    return files

report_names = [f for f in get_all_filenames(report_fir_path) if f.startswith(family)] # 只選擇這個家族的報告
csv_names = [f for f in get_all_filenames(report_csv_path) if f.startswith(family)]
report_dfs = [pd.read_csv(f"{report_csv_path}{f}") for f in csv_names]
report_txts = []
for f in report_names:
    with open(f"{report_fir_path}{f}", 'r', encoding='utf-8') as f:
        content = f.read()
        report_txts.append(content)
report_ids = {
    'Dofloo-BleepingComputer.txt': 'D1,1',
    'Dofloo-Securityaffairs.txt': 'D1,2',
    'Dofloo-SyscallParty.txt': 'D1,3',
    'Dofloo-Trendmicro.txt': 'D1,4',
}
report_names

['Dofloo-BleepingComputer.txt',
 'Dofloo-Securityaffairs.txt',
 'Dofloo-SyscallParty.txt',
 'Dofloo-Trendmicro.txt']

#### 產生答案，讓我線下人工標註

In [19]:
data = []

for df, name in zip(report_dfs, report_names):
    rid = report_ids[name]
    for i, row in df.iterrows():
        sent = row['Content'].replace('\n',' ')
        words_in_sent = sent.split()
        candidates = [w for w in words_in_sent if '/' in w or '.' in w.strip('.')]
        if len(candidates) == 0:
            continue
        for cand in candidates:
            a_row = {
               'rid': rid,
               'STobj': cand,
               'TrueAns': 1,
               'Extracted': 0,
               'sentence': sent,
            }
            data.append(a_row)
        # print(candidates)
    # break
df = pd.DataFrame(data)
# df.to_csv('./thesis_exp1_out.csv', index=False)

#### 產生 Extraction 模組的結果

In [26]:
extract_data = []
for txt, name in zip(report_txts, report_names):
    report_tf = {}
    for pattern in asg_regex + cntone_regex:
        rid = report_ids[name]
        matches = re.findall(pattern, txt)
        if matches == []:
            continue
        for match in matches:
            a_row = {
                'rid': rid[:15],
                'STobj': match,
                'regex': pattern,
                # 'TrueAns': 1,
                # 'Extracted': 0,
                # 'sentence': sent,
            }
            extract_data.append(a_row)
        # print(matches)
        # cnt = len(re.findall(pattern, txt))
        # report_tf[pattern] = cnt
dfext = pd.DataFrame(extract_data)
dfext.to_csv('./extracted.csv', index=False)
dfext

# extracted_strings = dfext['STobj'].unique()

,rid,STobj,regex
0,"D1,2","/etc/rc.local and /etc/rc.d/rc.local, as an au...",/etc/rc.*\.d/.*
1,"D1,2",/etc/rc.local,/etc/rc\.local
2,"D1,2",23.224.59.34,"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
3,"D1,2",23.224.59.34:48080,"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+"
4,"D1,3",/etc/init.d/boot.local files (in the auto_boot...,/etc/init\.d/.*
5,"D1,3",/etc/rc.local and the /etc/init.d/boot.local f...,/etc/rc.*\.d/.*
6,"D1,3",/etc/rc.local,/etc/rc\.local
7,"D1,3",/etc/rc.local,/etc/rc\.local
8,"D1,3",/etc/rc.local,/etc/rc\.local
9,"D1,3",/proc/net/dev,/proc/net/dev


#### 計算 TP rate (總攬 & 個別)
有 4 個 False Negetive，因為其他樣本有這個 STobj，但我們選用的沒有。是 /proc/cpuinfo, /etc/host.conf, /etc/resolv.conf, /etc/nsswitch.conf

In [47]:
df = pd.read_csv('./thesis_exp1.csv')
y_true = df['TrueAns']
y_pred = df['Extracted']
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
tpr = tp / (tp+fn)
print('tn, fp, fn, tp:', [tn, fp, fn, tp])
print('Manual:',sum(y_true),',Extracted:',sum(y_pred), ',TP rate:', tpr)
confusion_matrix(y_true, y_pred)

tn, fp, fn, tp: [23, 0, 5, 46]
Manual: 51 ,Extracted: 46 ,TP rate: 0.9019607843137255


array([[23,  0],
       [ 5, 46]], dtype=int64)

In [49]:
df[(df['TrueAns'] == 1) & (df['Extracted'] == 0)]

,rid,STobj,TrueAns,Extracted,sentence
22,"D1,3",/proc/cpuinfo,1,0,The virtual file /proc/cpuinfo is opened and r...
59,"D1,4",/proc/cpuinfo,1,0,This Backdoor gathers the following data: CP...
69,"D1,4",/etc/host.conf,1,0,This Backdoor opens the following files: /et...
70,"D1,4",/etc/resolv.conf,1,0,This Backdoor opens the following files: /et...
71,"D1,4",/etc/nsswitch.conf,1,0,This Backdoor opens the following files: /et...


In [43]:
# df1 = df[df['rid'] == 'D1,1'] # 沒有任何 STobj
df2 = df[df['rid'] == 'D1,2']
df3 = df[df['rid'] == 'D1,3']
df4 = df[df['rid'] == 'D1,4']
for i,df in enumerate([df2, df3, df4]):
    y_true = df['TrueAns']
    y_pred = df['Extracted']
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    tpr = tp / (tp+fn)
    print(f"D1,{i+2}",'Manual:',sum(y_true),',Extracted:',sum(y_pred), ',TP rate:', tpr)

D1,2 Manual: 4 ,Extracted: 4 ,TP rate: 1.0
D1,3 Manual: 31 ,Extracted: 30 ,TP rate: 0.967741935483871
D1,4 Manual: 16 ,Extracted: 12 ,TP rate: 0.75


In [11]:
txt = report_txts[2]

def get_metric(txt:str, reportname:str):
    # record appearance of regex
    report_tf = {}
    for pattern in asg_regex + cntone_regex:
        cnt = len(re.findall(pattern, txt))
        report_tf[pattern] = cnt

    # calculate the metrics
    score = 0
    score_sys = 0
    for pattern, cnt in report_tf.items():
        # print(pattern)
        if pattern in cntone_regex and cnt:
            score += 1
        else:
            score += cnt
        if pattern.find('/etc') >= 0:
            if pattern in cntone_regex:
                continue
            score_sys += cnt
    print(f"{reportname:30} score: {score:2}, score_sys: {score_sys}")
    return report_tf

results = []
for txt, name in zip(xor_report_txts, xor_report_names):
    report_tf = get_metric(txt, name)
    results.append(report_tf)

Dofloo-BleepingComputer.txt    score:  0, score_sys: 0
Dofloo-Securityaffairs.txt     score:  4, score_sys: 2
Dofloo-SyscallParty.txt        score:  9, score_sys: 5
Dofloo-Trendmicro.txt          score: 12, score_sys: 6


In [12]:
results[0]

{'.*/selinux': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/init\\.d/.*': 0,
 '/etc/rc.*\\.d/.*': 0,
 '/etc/rc\\.local': 0,
 '/proc/net/dev': 0,
 '/proc/self': 0,
 '/proc/stat': 0,
 '/sys/devices/.*': 0,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 0}

In [13]:
results[1]

{'.*/selinux': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/init\\.d/.*': 0,
 '/etc/rc.*\\.d/.*': 1,
 '/etc/rc\\.local': 1,
 '/proc/net/dev': 0,
 '/proc/self': 0,
 '/proc/stat': 0,
 '/sys/devices/.*': 0,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 1,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 1}

In [14]:
results[2]

{'.*/selinux': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/init\\.d/.*': 1,
 '/etc/rc.*\\.d/.*': 1,
 '/etc/rc\\.local': 3,
 '/proc/net/dev': 1,
 '/proc/self': 1,
 '/proc/stat': 0,
 '/sys/devices/.*': 0,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 20,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 1,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 0}

In [15]:
results[3]

{'.*/selinux': 0,
 '/.*bin/sed': 0,
 '/dev/urandom': 0,
 '/etc/init\\.d/.*': 1,
 '/etc/rc.*\\.d/.*': 1,
 '/etc/rc\\.local': 4,
 '/proc/net/dev': 1,
 '/proc/self': 4,
 '/proc/stat': 1,
 '/sys/devices/.*': 0,
 '/etc/sed[0-9a-zA-Z]{6}': 0,
 'Permission': 0,
 'Permission:[0-9]{3}': 0,
 '^GID.*': 0,
 '^PID.*': 0,
 '^UID.*': 0,
 '^eth.*': 0,
 '^sed$': 0,
 '^sh$': 0,
 '^uname$': 0,
 '0x[0-9a-zA-Z]{1,16}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': 0,
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': 0}